In [2]:
%%capture
from photozpy import *
from ccdproc import ImageFileCollection
from pathlib import Path
from astropy.time import Time
import numpy as np

from astropy.nddata import CCDData
from astropy.visualization import hist
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from astropy.stats import mad_std

#from convenience_functions import show_image
from astropy.io import fits
#plt.style.use('guide.mplstyle')

from astropy.io import fits

from photozpy import CollectionManager
from photozpy import SourceDetection
from photozpy import PlateSolving
from photozpy import Sources
from photozpy import Photometry

from photutils.centroids import centroid_quadratic, centroid_com, centroid_sources, centroid_2dg

from astropy.coordinates import SkyCoord
import astropy.units as u

# Pretreat the fits images
- Links the object names with the the name of fits files.
- Add necessary headers and header values

Download the tutorial data from this [link](https://drive.google.com/drive/folders/1zwNlXxlR6BPgA8OvVF8YS5HOREVSonot?usp=sharing)

In [5]:
# define the telescope
# It handles the telescope names, filters, ccd type, ccd gain and ccd readnoise
telescope = Telescope(telescope = "CT", mode = "griz", ccd = "Andor")

In [4]:
# get the original 
path = Path("/Users/shengyong/conda_envs/photozpy/photozpy/tests/20231230_Examined_has_flats")
image_collection = ImageFileCollection(location = path, glob_include = "*.fits")

In [4]:
# map the fits file name to the object names
# this will be handled by Sources module
targets = {"NAME_Vel_G": "NAME Vel G",
           "E1_b": "E1 b"}

In [6]:
# create a working folder and correct the image header
working_folder = "/Users/shengyong/conda_envs/photozpy/photozpy/tests/20231230"
header_edition = HeaderCorrection(image_collection = image_collection, telescope = telescope, target_dict = targets)

header_edition.correct_headers_by_filename(save_location = Path(working_folder))
header_edition.correct_filter_headers()

Editing headers by file names......


100%|█████████████████████████████████████████| 216/216 [00:19<00:00, 11.04it/s]


Header edition by file names completed!
----------------------------------------

Editting filter headers......


156it [00:07, 20.93it/s]


Filter header edition completed!
----------------------------------------



In [5]:
# set up the working path
working_folder = "/Users/shengyong/conda_envs/photozpy/photozpy/tests/20231230"
path = Path(working_folder)
working_image_collection = ImageFileCollection(location = path, glob_include = "*.fits")

# Calibration Correction, Registration, Plate Solving and Combine

In [4]:
# define calibration objects
combine_images = Combine(working_image_collection, telescope)
calibrate_images = Reduction(working_image_collection, telescope)
registration = Registration(working_image_collection, telescope)
sources_detection = SourceDetection(working_image_collection)
plate_solving = PlateSolving(working_image_collection)

## Bias Correction

In [9]:
# Combine Bias
combine_images.combine_bias_or_dark(image_type = "Bias")

# Apply Bias correction
calibrate_images.apply_bias_correction()

Using ['Bias_009.fits', 'Bias_025.fits', 'Bias_002.fits', 'Bias_010.fits', 'Bias_011.fits', 'Bias_004.fits', 'Bias_024.fits', 'Bias_027.fits', 'Bias_017.fits', 'Bias_019.fits', 'Bias_001.fits', 'Bias_026.fits', 'Bias_005.fits', 'Bias_029.fits', 'Bias_008.fits', 'Bias_003.fits', 'Bias_006.fits', 'Bias_013.fits', 'Bias_018.fits', 'Bias_020.fits', 'Bias_007.fits', 'Bias_014.fits', 'Bias_016.fits', 'Bias_021.fits', 'Bias_030.fits', 'Bias_022.fits', 'Bias_028.fits', 'Bias_015.fits', 'Bias_012.fits', 'Bias_023.fits'] to combine the master Bias image!
Bias combined!
----------------------------------------

Apply Master_Bias.fits correction to Flat_SDSS_r_057.fits
Apply Master_Bias.fits correction to Dark_019.fits
Apply Master_Bias.fits correction to Flat_SDSS_g_009.fits
Apply Master_Bias.fits correction to Dark_029.fits
Apply Master_Bias.fits correction to Flat_SDSS_g_016.fits
Apply Master_Bias.fits correction to Flat_SDSS_g_035.fits
Apply Master_Bias.fits correction to Flat_SDSS_z_014.fits


## Dark Correction

In [10]:
# Combine Dark
combine_images.combine_bias_or_dark(image_type = "dark")

# Apply Dark correction
calibrate_images.apply_dark_correction()

Using ['Dark_007.fits', 'Dark_019.fits', 'Dark_029.fits', 'Dark_024.fits', 'Dark_010.fits', 'Dark_012.fits', 'Dark_005.fits', 'Dark_023.fits', 'Dark_004.fits', 'Dark_015.fits', 'Dark_016.fits', 'Dark_011.fits', 'Dark_020.fits', 'Dark_013.fits', 'Dark_025.fits', 'Dark_022.fits', 'Dark_009.fits', 'Dark_030.fits', 'Dark_018.fits', 'Dark_021.fits', 'Dark_028.fits', 'Dark_008.fits', 'Dark_027.fits', 'Dark_003.fits', 'Dark_017.fits', 'Dark_006.fits', 'Dark_026.fits', 'Dark_001.fits', 'Dark_014.fits', 'Dark_002.fits'] to combine the master Dark image!
dark combined!
----------------------------------------

Apply Master_Dark.fits correction to Flat_SDSS_r_057.fits
Apply Master_Dark.fits correction to Flat_SDSS_g_009.fits
Apply Master_Dark.fits correction to Flat_SDSS_g_016.fits
Apply Master_Dark.fits correction to Flat_SDSS_g_035.fits
Apply Master_Dark.fits correction to Flat_SDSS_z_014.fits
Apply Master_Dark.fits correction to NAME_Vel_G_SDSS_g_053.fits
Apply Master_Dark.fits correction to F

## Flat Correction

In [11]:
# Combine Flat
combine_images.combine_light_or_flat(image_type = "Flat")

# Apply Flat Correction
calibrate_images.apply_flat_correction()

Using ['Flat_SDSS_g_009.fits', 'Flat_SDSS_g_035.fits', 'Flat_SDSS_g_016.fits', 'Flat_SDSS_g_004.fits', 'Flat_SDSS_g_056.fits', 'Flat_SDSS_g_007.fits', 'Flat_SDSS_g_021.fits', 'Flat_SDSS_g_022.fits', 'Flat_SDSS_g_028.fits', 'Flat_SDSS_g_061.fits', 'Flat_SDSS_g_050.fits', 'Flat_SDSS_g_040.fits', 'Flat_SDSS_g_055.fits', 'Flat_SDSS_g_008.fits', 'Flat_SDSS_g_023.fits', 'Flat_SDSS_g_014.fits', 'Flat_SDSS_g_045.fits', 'Flat_SDSS_g_015.fits', 'Flat_SDSS_g_029.fits', 'Flat_SDSS_g_030.fits'] to combine the master Flat image in SDSS_g' filter!
Flat in SDSS_g' filter combined!
----------------------------------------------------------

Using ['Flat_SDSS_r_057.fits', 'Flat_SDSS_r_052.fits', 'Flat_SDSS_r_046.fits', 'Flat_SDSS_r_037.fits', 'Flat_SDSS_r_041.fits', 'Flat_SDSS_r_058.fits', 'Flat_SDSS_r_038.fits', 'Flat_SDSS_r_003.fits', 'Flat_SDSS_r_048.fits', 'Flat_SDSS_r_051.fits', 'Flat_SDSS_r_035.fits', 'Flat_SDSS_r_053.fits', 'Flat_SDSS_r_059.fits', 'Flat_SDSS_r_062.fits', 'Flat_SDSS_r_047.fits', '

## Register source images

In [15]:
# Register source images
registration.Register_images(filter = telescope.filters[2])

Aligning ['NAME Vel G'] ......
Aligning NAME_Vel_G_SDSS_z_016.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_g_033.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_g_021.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_g_009.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_g_037.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_g_005.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_r_026.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_r_038.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_z_036.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_r_058.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_i_031.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_i_019.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_z_056.fits using NAME_Vel_G_SDSS_i_003.fits
Aligning NAME_Vel_G_SDSS_g_029.fits using NAME_Vel_G_SDSS_i_003.fits
Ali

## Combine Light images

In [17]:
# combine light images
combine_images.combine_light_or_flat(image_type = "Light")

Using ['NAME_Vel_G_SDSS_g_033.fits', 'NAME_Vel_G_SDSS_g_013.fits', 'NAME_Vel_G_SDSS_g_021.fits', 'NAME_Vel_G_SDSS_g_009.fits', 'NAME_Vel_G_SDSS_g_037.fits', 'NAME_Vel_G_SDSS_g_005.fits', 'NAME_Vel_G_SDSS_g_025.fits', 'NAME_Vel_G_SDSS_g_045.fits', 'NAME_Vel_G_SDSS_g_017.fits', 'NAME_Vel_G_SDSS_g_049.fits', 'NAME_Vel_G_SDSS_g_001.fits', 'NAME_Vel_G_SDSS_g_057.fits', 'NAME_Vel_G_SDSS_g_029.fits', 'NAME_Vel_G_SDSS_g_041.fits', 'NAME_Vel_G_SDSS_g_053.fits'] to combine the master NAME Vel G image in SDSS_g' filter!
NAME Vel G in SDSS_g' filter combined!
----------------------------------------------------------

Using ['NAME_Vel_G_SDSS_r_038.fits', 'NAME_Vel_G_SDSS_r_018.fits', 'NAME_Vel_G_SDSS_r_042.fits', 'NAME_Vel_G_SDSS_r_026.fits', 'NAME_Vel_G_SDSS_r_022.fits', 'NAME_Vel_G_SDSS_r_058.fits', 'NAME_Vel_G_SDSS_r_030.fits', 'NAME_Vel_G_SDSS_r_034.fits', 'NAME_Vel_G_SDSS_r_050.fits', 'NAME_Vel_G_SDSS_r_054.fits', 'NAME_Vel_G_SDSS_r_002.fits', 'NAME_Vel_G_SDSS_r_006.fits', 'NAME_Vel_G_SDSS_r_

## Plate solving

In [5]:
# beofore plate solving, we need to find the FWHM of the PSF first
sources_detection.find_fwhm(fit_box_size = 301)

# Add WCS to the Master Light images
# If the Master Light images contains WCS, it will be overwritten
plate_solving.add_wcs(api = "mmzlynroxjblhdzw")

Determining the local peaks of E1 b in SDSS_i' filter.....
Determining the centroids of E1 b in SDSS_i' filter.....
----------------------------------------

Determining the local peaks of NAME Vel G in SDSS_i' filter.....
Determining the centroids of NAME Vel G in SDSS_i' filter.....
----------------------------------------

Determining the local peaks of NAME Vel G in SDSS_z' filter.....
Determining the centroids of NAME Vel G in SDSS_z' filter.....
----------------------------------------

Determining the local peaks of E1 b in SDSS_r' filter.....
Determining the centroids of E1 b in SDSS_r' filter.....
----------------------------------------

Determining the local peaks of NAME Vel G in SDSS_g' filter.....
Determining the centroids of NAME Vel G in SDSS_g' filter.....
----------------------------------------

Determining the local peaks of NAME Vel G in SDSS_r' filter.....
Determining the centroids of NAME Vel G in SDSS_r' filter.....
----------------------------------------

Dete

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Determining background stats
Finding sources
Found 119 sources
 id     xcentroid      ...        flux                 mag          
--- ------------------ ... ------------------ ----------------------
 37  359.4472405455978 ... 2082.7744449453266      -8.29660560091403
111  96.56611833525365 ...  584.2198994032223     -6.916440864022054
 93 1045.0284710312521 ... 399.25343017843835     -6.503121640148834
 66 204.39135935243337 ... 233.97345862739004     -5.922916487270804
 88  2009.319357136736 ... 191.52580574093744     -5.705568245185031
 85  32.60057291344423 ... 165.98623441365382     -5.550180181454344
117  1285.715892613958 ...  162.2913732862531     -5.525738588016745
...                ... ...                ...                    ...
 50 1817.4980643390102 ...  1.028307064514136   -0.03030704859172579
 62 1637.9850233128288 ... 1.0248883130825281  -0.0266913521

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


WCS added to Master_E1 b_SDSS_i'.fits
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Determining background stats
Finding sources
Found 2163 sources
 id      xcentroid      ...        flux                 mag          
---- ------------------ ... ------------------ ----------------------
1328  1002.337944220714 ...  3630.515497351652     -8.899920737356325
1809 1879.9869069894453 ...  3292.935844798826      -8.79395817393945
1529  882.3532688875562 ...  3029.375922161341     -8.703382923172159
 946  584.0801679837092 ...  2588.228255902328     -8.532506435311946
1160  355.2862370662413 ... 1973.2694557371765     -8.237965983720294
  18  301.5893163066814 ...  1967.556084857714      -8.23481780174811
 804  1121.446485629143 ... 1592.2532479253086     -8.005030358613357
 ...                ... ...                ...                    ...
 359  99.73001948540413 ... 1.0027038930671905  -0.002931752807566652
1978 1610.

...

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


WCS added to Master_NAME Vel G_SDSS_r'.fits
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Determining background stats
Finding sources
Found 78 sources
 id     xcentroid      ...        flux                 mag         
--- ------------------ ... ------------------ ---------------------
 27  359.5091487066465 ...  514.1121876084924    -6.777644748572355
 70  96.61427216783535 ... 152.92980399962482     -5.46123032980311
 63 1045.1355546352713 ... 109.33118840221645     -5.09686017200247
 39 204.35712627692882 ...   61.3613346919281   -4.4697369938384295
 59  2009.387996144458 ... 49.391024456799286    -4.234120085717733
 77  1285.789983688648 ...  42.85351623269866    -4.079966156535214
 57 32.553003013457115 ...  40.84126501796733    -4.027747963667827
...                ... ...                ...                   ...
 51  819.6076941237509 ... 1.1929343268122656   -0.1915413390834244
 62  734.6115378883355 ... 1

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


WCS added to Master_E1 b_SDSS_z'.fits
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Determining background stats
Finding sources
Found 141 sources
 id     xcentroid      ...        flux                 mag         
--- ------------------ ... ------------------ ---------------------
 47  359.5440597336397 ...  2363.326099721988    -8.433809127968491
124  96.71901369308168 ...  460.1999245242801    -6.657366357763921
106 1045.2601055744517 ...  262.2699309046919    -6.046871254585549
 75 204.58525172038543 ...  196.6008243995568    -5.733963336530573
 97  32.68347104890015 ... 161.04985775041874    -5.517400863897728
101 2009.5906946107266 ... 159.15754841511819     -5.50456810216624
140 1286.0895671513283 ... 130.12469664601966    -5.285899325170385
...                ... ...                ...                   ...
  7  1516.608141893562 ... 1.0178031649729562  -0.01915949256679233
122 2032.3037718234164 ...  1.015

WCS added to Master_E1 b_SDSS_g'.fits


# Photometry

Here let me explain the difference between sources and objects:
1. **Object**: this is used to identify the images. When you observe, you input the object name in the observation software. It can be the same as source or not, depending on if there are one or multiple sources in the image.
2. **Source**: this is used to identify each individual sources in each image.
For example, we use the object name *E1 b* to identify the standard star images, while there are multiple standard star sources in a standard image.

## Define sources

In [6]:
objects = ["E1 b", "NAME Vel G"]

skycoord_E1 = SkyCoord(ra = ["1:24:27.0792", "1:24:08.6412", "1:24:00.1725"], 
                       dec = ["-44:18:31.360", "-44:19:52.773", "-44:17:30.642"], 
                       frame='icrs',unit=(u.hourangle, u.deg))

skycoord_NAME = SkyCoord(ra = ["08:33:12.4538"],
                         dec = ["-44:40:39.830"],
                         frame='icrs',unit=(u.hourangle, u.deg))

sources = Sources(object = objects, skycoords = [skycoord_E1, skycoord_NAME])

## Start Photometry

In [7]:
# define Photometry object
photometry = Photometry(working_image_collection)

In [11]:
photometry.aperture_photometry(sources = sources)
# Note that the output are the uncalibrated magnitudes.
# The calibration can be done easily now manually by the standard stars.
# This calibration will be handled and automated by the `standard_stars` module.
# But at this point, the purpose of photometry provided by `photozpy` is completed and ready for more tests.
# Other functions like plotting, cosmic ray rejection, saturation detection, automated calibration will be completed
# as the optimization of `photozpy` in the future while analzing more data.

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


Working on photometry of E1 b in SDSS_i' ......
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Conversion from sky coordiantes to pixel coordinates completed!


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


 id  xcenter   ycenter  ... total_bkg aperture_sum_bkgsub  mag_inst 
       pix       pix    ...    adu            adu                   
--- --------- --------- ... --------- ------------------- ----------
  1 576.63822 1118.1153 ... 45672.131            64150.92 -12.018007
  2 1156.4399 1352.1582 ... 45443.307           64952.195 -12.031485
  3 1420.2181 935.51577 ... 45739.443           61146.162 -11.965923
----------------------------------------------------------

Working on photometry of E1 b in SDSS_g' ......
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Conversion from sky coordiantes to pixel coordinates completed!


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

 id  xcenter   ycenter  ... total_bkg aperture_sum_bkgsub  mag_inst 
       pix       pix    ...    adu            adu                   
--- --------- --------- ... --------- ------------------- ----------
  1 1233.2602 1307.9084 ... 105582.04           45222.059 -11.638376
----------------------------------------------------------

Working on photometry of NAME Vel G in SDSS_z' ......
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Conversion from sky coordiantes to pixel coordinates completed!


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


 id  xcenter   ycenter  ... total_bkg aperture_sum_bkgsub  mag_inst
       pix       pix    ...    adu            adu                  
--- --------- --------- ... --------- ------------------- ---------
  1 1233.4883 1307.8344 ... 48227.601           31478.944 -11.24505
----------------------------------------------------------

Working on photometry of NAME Vel G in SDSS_g' ......
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Conversion from sky coordiantes to pixel coordinates completed!


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


 id  xcenter   ycenter  ... total_bkg aperture_sum_bkgsub  mag_inst 
       pix       pix    ...    adu            adu                   
--- --------- --------- ... --------- ------------------- ----------
  1 1233.2274 1307.9335 ...  106808.9           10985.621 -10.102062
----------------------------------------------------------

Working on photometry of NAME Vel G in SDSS_i' ......
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Conversion from sky coordiantes to pixel coordinates completed!
 id  xcenter  ycenter aperture_sum ... total_bkg aperture_sum_bkgsub  mag_inst 
       pix      pix       adu      ...    adu            adu                   
--- --------- ------- ------------ ... --------- ------------------- ----------
  1 1233.4099 1307.83    141602.45 ... 79971.566           61630.885 -11.974496
----------------------------------------------------------

